## Load packages and initialize GEE

In [97]:
# import packages
import ee
import geemap

import pandas as pd

In [98]:
# initialize the EE api
ee.Initialize(project='ee-bermane')

## Define GEE datasets

In [99]:
# define EE datasets
agb = ee.ImageCollection("projects/sat-io/open-datasets/ESA/ESA_CCI_AGB")
y2y = ee.FeatureCollection("projects/ee-bermane/assets/y2y")
lc = ee.Image("USGS/NLCD_RELEASES/2020_REL/NALCMS")
rsr = ee.Image("projects/ee-bermane/assets/Root_shoot_ratio_Map_Merged")
spawn = ee.ImageCollection("NASA/ORNL/biomass_carbon_density/v1")
countries = ee.FeatureCollection("USDOS/LSIB/2017")
y2y_ecoregions = ee.FeatureCollection(
    "projects/ee-bermane/assets/y2y_ecoregions")
y2y_biomes = ee.FeatureCollection("projects/ee-bermane/assets/y2y_biomes")
y2y_protected_areas = ee.FeatureCollection(
    "projects/ee-bermane/assets/y2y_protected_areas")
soc_olm = ee.Image("projects/ee-bermane/assets/soc_0_1m_kg_m2_olm")
soc_soilgrids = ee.Image("projects/soilgrids-isric/ocs_mean")
soc_sothe = ee.Image(
    "projects/ee-bermane/assets/McMaster_WWFCanada_soil_carbon1m_250m_kg-m2_version3")
soc_socs = ee.Image("projects/ee-bermane/assets/SOCS_0_1m_mg_ha_year_2010")

## Define functions

In [110]:
# Define function to calculate zonal statistics
# specify image to use and reducer type (sum(), mean())
def calc_zonal_stats(feature, image_list, reducer_list):

    # loop through image and reducer list
    for image, reducer in zip(image_list, reducer_list):

        # calculate zonal stats across features
        stats = image.reduceRegion(
            reducer=reducer,
            geometry=feature.geometry(),
            crs=image.select(0).projection(),
            crsTransform=image.select(0).projection().getInfo().get('transform'),
            maxPixels=1e30
        )

        # add zonal stats to feature properties
        feature = feature.set(stats)

    return feature

# Define function to calculate feature area in km2

# def calc_feature_area(feature):
#     area_km2 = feature.geometry().area().divide(1e6)
#     return feature.set({'area_km2_gee': area_km2})

# define function to calculate additional carbon stock values
# since pixel sizes vary instead of taking the mean of all density values from the rasters
# we are calculating the total carbon and dividing it by the area in ha
def calc_addl_stats(df):
    # calc total biomass
    df['tob_t'] = df['agb_t'] + df['bgb_t'] + df ['dpm_t']

    # calc densities
    df['agb_t_ha'] = df['agb_t'] / df['pixel_area_agb_extent_ha']
    df['bgb_t_ha'] = df['bgb_t'] / df['pixel_area_agb_extent_ha']
    df['dpm_t_ha'] = df['dpm_t'] / df['pixel_area_agb_extent_ha']
    df['tob_t_ha'] = df['tob_t'] / df['pixel_area_agb_extent_ha']
    df['soc_t_ha'] = df['soc_t'] / df['pixel_area_soc_extent_ha']

    # calc % of storage
    df['perc_tob'] = df['tob_t'] / df['tob_t'].sum()
    df['perc_soc'] = df['soc_t'] / df['soc_t'].sum()

    # calc % of area
    df['perc_tob_extent'] = df['pixel_area_agb_extent_ha'] / df['pixel_area_agb_extent_ha'].sum()
    df['perc_soc_extent'] = df['pixel_area_soc_extent_ha'] / df['pixel_area_soc_extent_ha'].sum()
    df['perc_area'] = df['pixel_area_ha'] / df['pixel_area_ha'].sum()

    # return df
    return df

## Calculate biomass stock from ESA CCI data

In [143]:
# calc 2021 ESA CCI biomass
# grab the 2021 AGB images
agb_2021 = agb.filter(ee.Filter.stringContains("system:index", "2021")).first()

# add BGB to 2021 image using global rsr map
bio_2021 = agb_2021.addBands(agb_2021.select(['AGB']).multiply(rsr).rename('BGB'))

# create image for DPM (dead wood + litter) using Harris ratio (8% + 4%)
# mask DPM only for forested LC types
# 1: "#033e00",  # Temperate or sub-polar needleleaf forest
# 2: "#939b71",  # Sub-polar taiga needleleaf forest
# 3: "#196d12",  # Tropical or sub-tropical broadleaf evergreen forest
# 4: "#1fab01",  # Tropical or sub-tropical broadleaf deciduous forest
# 5: "#5b725c",  # Temperate or sub-polar broadleaf deciduous forest
# 6: "#6b7d2c",  # Mixed forest
dpm_mask = lc.gte(1).And(lc.lte(6))

dpm = agb_2021.select(['AGB']).multiply(0.12).updateMask(dpm_mask).rename('DPM')

# add DPM to 2021 image and rename bands
bio_2021 = bio_2021.addBands(dpm).select(['AGB', 'BGB', 'DPM']).rename(['agb_t_ha', 'bgb_t_ha', 'dpm_t_ha'])

# multiply values by 0.47 to get carbon density
# 0.47 used by Harris et al. (2021)
bio_2021 = bio_2021.multiply(0.47).updateMask(lc.neq(18).And(lc.neq(19).And(lc.neq(16))))

# compute per-pixel area in ha
pixel_area_ha = ee.Image.pixelArea().divide(10000)

# Create carbon layer mask to filter pixel area raster
carbon_mask = bio_2021.select(['agb_t_ha']).mask().neq(0)

# Mask pixel_area_ha to carbon layers
pixel_area_agb_extent = pixel_area_ha.updateMask(carbon_mask)

# calculate total biomass c per pixel and rename bands
bio_stock_2021 = bio_2021.multiply(
    pixel_area_ha
).rename(
    ['agb_t', 'bgb_t', 'dpm_t']
).addBands(
    pixel_area_agb_extent.rename(['pixel_area_agb_extent_ha'])
).addBands(
    pixel_area_ha.rename(['pixel_area_ha'])
)

## Calculate soil carbon from Sothe Canada data and Open Land Map US data

In [113]:
# calc open land map soc
# multiply by 10 to get t/ha
# mask water/snow/ice
soc_olm = soc_olm.multiply(10).rename(
    'soc_dens').updateMask(lc.neq(18).And(lc.neq(19)))

# multiply by pixel area to get total carbon per pixel
soc_olm_stock = soc_olm.multiply(pixel_area_ha).rename('total_olm_soc')

# reproject to match sothe to blend images
soc_olm_reproj_sothe = soc_olm.resample('bilinear').toFloat()

In [114]:
# calc sothe soc
# multiply by 10 to get t/ha
# mask water/snow/ice
soc_sothe = soc_sothe.multiply(10).rename(
    'soc_dens').updateMask(lc.neq(18).And(lc.neq(19)))

# blend sothe and olm carbon across y2y
soc_blend = ee.ImageCollection([soc_olm_reproj_sothe, soc_sothe]).mosaic().rename('soc_t_ha').reproject(
    crs=soc_sothe.projection(),
    crsTransform=soc_sothe.projection().getInfo().get('transform')
).updateMask(lc.neq(18).And(lc.neq(19)))

# Create soc layer mask to filter pixel area raster
soc_mask = soc_blend.mask().neq(0)

# Mask pixel_area_ha to carbon layers
pixel_area_soc_extent = pixel_area_ha.updateMask(soc_mask)

# multiply by pixel area to get total carbon per pixel
soc_blend_stock = soc_blend.multiply(pixel_area_ha).rename('soc_t').addBands(
    pixel_area_soc_extent.rename(['pixel_area_soc_extent_ha'])
).addBands(
    pixel_area_ha.rename(['pixel_area_ha'])
)

In [115]:
# # create a map
# m = geemap.Map()

# # add layers
# # m.addLayer(soc_blend, {"min": 1, "max": 450, "palette": biomass_palette}, "SOC Density Blend 0-1m")
# m.addLayer(soc_blend.clip(y2y), {}, 'SOC Density')
# m.addLayer(pixel_area_soc_extent.clip(y2y), {}, 'SOC Mask')

# # Display the map
# m

In [116]:
# # calc socs soc
# # mask water/snow/ice
# soc_socs = soc_socs.rename('soc_socs_t_ha').updateMask(
#     lc.neq(18).And(lc.neq(19)))

# # Create soc layer mask to filter pixel area raster
# socs_mask = soc_socs.mask().neq(0)

# # Mask pixel_area_ha to carbon layers
# pixel_area_socs_extent = pixel_area_ha.updateMask(socs_mask)

# # multiply by pixel area to get total carbon per pixel
# soc_socs_stock = soc_socs.multiply(pixel_area_ha).rename('soc_socs_t').addBands(
#     pixel_area_socs_extent.rename(['pixel_area_model_extent_ha'])
# ).addBands(
#     pixel_area_ha
# )

## Check scale of output layers

In [117]:
# check scale of layers
print('Biomass Carbon Density Scale: ',
      bio_2021.projection().nominalScale().getInfo())
print('Biomass Carbon Total Scale: ',
      bio_stock_2021.select(['agb_t']).projection().nominalScale().getInfo())
#print('SOC Density Sothe Scale: ', soc_sothe.projection().nominalScale().getInfo())
print('SOC Density Blend Scale: ', soc_blend.select(['soc_t_ha']).projection().nominalScale().getInfo())
print('SOC Total Blend Scale: ',
      soc_blend_stock.select(['soc_t']).projection().nominalScale().getInfo())
#print('SOC Density OLM Scale: ', soc_olm.projection().nominalScale().getInfo())

Biomass Carbon Density Scale:  98.95065848290943
Biomass Carbon Total Scale:  98.95065848290943
SOC Density Blend Scale:  250.00021298953834
SOC Total Blend Scale:  250.00021298953834


## Compute zonal statistics and convert results to dataframe

In [118]:
# define image and reducer list for zonal stats
img_list = [bio_stock_2021, soc_blend_stock]
redu_list = [ee.Reducer.sum(), ee.Reducer.sum()]

# define output folder for exports
out_folder = './outputs/'

# Compute zonal statistics across Y2Y region
y2y_carbon = y2y.map(lambda feature: calc_zonal_stats(
    feature, img_list, redu_list))

# convert to df
y2y_df = ee.data.computeFeatures({
    'expression': y2y_carbon,
    'fileFormat': 'PANDAS_DATAFRAME'
})

# add density values
y2y_df = calc_addl_stats(y2y_df)

# drop geometry and export as csv
y2y_df.drop('geo', axis=1).to_csv(out_folder + 'y2y_carbon_stocks.csv', index=False)

In [119]:
# Compute zonal statistics across biomes
biome_carbon = y2y_biomes.map(lambda feature: calc_zonal_stats(
    feature, img_list, redu_list))

# convert to df
biome_df = ee.data.computeFeatures({
    'expression': biome_carbon,
    'fileFormat': 'PANDAS_DATAFRAME'
})

# add density values
biome_df = calc_addl_stats(biome_df)

# drop geometry and export as csv
biome_df.drop('geo', axis=1).to_csv(out_folder + 'y2y_biome_carbon_stocks.csv', index=False)


In [ ]:
# Compute zonal statistics across ecoregions
eco_carbon = y2y_ecoregions.map(lambda feature: calc_zonal_stats(
    feature, img_list, redu_list))

# Compute zonal statistics across biomes
biome_carbon = y2y_biomes.map(lambda feature: calc_zonal_stats(
    feature, img_list, redu_list))

# Compute zonal statistics across protected areas
protected_areas_carbon = y2y_protected_areas.map(lambda feature: calc_zonal_stats(
    feature, img_list, redu_list))

# get canada and y2y geometry
canada = ee.FeatureCollection("USDOS/LSIB/2017").filter(ee.Filter.eq('COUNTRY_NA', 'Canada'))
can_geo = canada.geometry()
y2y_geo = y2y.geometry()

# get intersection of geometries
y2y_can_geo = y2y_geo.intersection(can_geo)

# create feature collection
y2y_can = ee.FeatureCollection(y2y_can_geo)

# Compute zonal statistics across y2y in canada
y2y_can_carbon = y2y.map(lambda feature: calc_zonal_stats(
    feature, img_list, redu_list))

# # Compute zonal statistics across Canada
# canada = countries.filter(ee.Filter.eq('COUNTRY_NA', 'Canada'))
# # img_list_can = [bio_stock_2021, bio_2021, soc_blend, soc_socs]
# # redu_list_can = [ee.Reducer.sum(), ee.Reducer.mean(), ee.Reducer.sum(),ee.Reducer.mean(), ee.Reducer.mean(), ee.Reducer.mean()]
# img_list_can = [bio_stock_2021]
# redu_list_can = [ee.Reducer.sum()]
# canada_carbon = canada.map(lambda feature

In [15]:
# export results to google drive
ee.batch.Export.table.toDrive(
    collection=y2y_carbon,
    description="y2y_carbon_stock",
    folder="carbon_stock_outputs",
    fileFormat="CSV"
).start()

# ee.batch.Export.table.toDrive(
#     collection=eco_carbon,
#     description="y2y_ecoregions_carbon_stock",
#     folder="carbon_stock_outputs",
#     fileFormat="CSV"
# ).start()

# ee.batch.Export.table.toDrive(
#     collection=biome_carbon,
#     description="y2y_biomes_carbon_stock",
#     folder="carbon_stock_outputs",
#     fileFormat="CSV"
# ).start()

# ee.batch.Export.table.toDrive(
#     collection=protected_areas_carbon,
#     description="y2y_protected_areas_carbon_stock",
#     folder="carbon_stock_outputs",
#     fileFormat="CSV"
# ).start()

# ee.batch.Export.table.toDrive(
#     collection=y2y_can_carbon,
#     description="y2y_canada_carbon_stock",
#     folder="carbon_stock_outputs",
#     fileFormat="CSV"
# ).start()

# ee.batch.Export.table.toDrive(
#     collection=canada_carbon,
#     description="canada_carbon_stock",
#     folder="carbon_stock_outputs",
#     fileFormat="CSV"
# ).start()

## Add layers to map

In [127]:
# set biomass palette
biomass_palette = [
    "#C6ECAE", "#A1D490", "#7CB970", "#57A751", "#348E32",
    "#267A29", "#176520", "#0C4E15", "#07320D", "#031807"
]

# set land cover palette
lc_palette = [
    "#033e00",  # Temperate or sub-polar needleleaf forest
    "#939b71",  # Sub-polar taiga needleleaf forest
    "#196d12",  # Tropical or sub-tropical broadleaf evergreen forest
    "#1fab01",  # Tropical or sub-tropical broadleaf deciduous forest
    "#5b725c",  # Temperate or sub-polar broadleaf deciduous forest
    "#6b7d2c",  # Mixed forest
    "#b29d29",  # Tropical or sub-tropical shrubland
    "#b48833",  # Temperate or sub-polar shrubland
    "#e9da5d",  # Tropical or sub-tropical grassland
    "#e0cd88",  # Temperate or sub-polar grassland
    "#a07451",  # Sub-polar or polar shrubland-lichen-moss
    "#bad292",  # Sub-polar or polar grassland-lichen-moss
    "#3f8970",  # Sub-polar or polar barren-lichen-moss
    "#6ca289",  # Wetland
    "#e6ad6a",  # Cropland
    "#a9abae",  # Barren land
    "#db2126",  # Urban and built-up
    "#4c73a1",  # Water
    "#fff7fe",  # Snow and ice
]

In [139]:
spawn

Name,Description
agb,"Aboveground living biomass carbon stock density of combined woody and herbaceous cover in 2010. This includes carbon stored in living plant tissues that are located above the earth’s surface (stems, bark, branches, twigs). This does not include leaf litter or coarse woody debris that were once attached to living plants but have since been deposited and are no longer living."
agb_uncertainty,Uncertainty of estimated aboveground living biomass carbon density of combined woody and herbaceous cover in 2010. Uncertainty represents the cumulative standard error that has been propagated through the harmonization process using summation in quadrature.
bgb,"Belowground living biomass carbon stock density of combined woody and herbaceous cover in 2010. This includes carbon stored in living plant tissues that are located below the earth’s surface (roots). This does not include dead and/or dislocated root tissue, nor does it include soil organic matter."
bgb_uncertainty,Uncertainty of estimated belowground living biomass carbon density of combined woody and herbaceous cover in 2010. Uncertainty represents the cumulative standard error that has been propagated through the harmonization process using summation in quadrature.


In [144]:
# create a map
m = geemap.Map()

# add layers
# m.addLayer(soc_blend, {"min": 1, "max": 450, "palette": biomass_palette}, "SOC Density Blend 0-1m")
# m.addLayer(rsr, {}, 'Root-to-Shoot Ratio Global')
m.addLayer(bio_2021.select(['agb_t_ha']).clip(y2y), {"min": 1, "max": 100, "palette": biomass_palette}, 'CCI AGB Density')
m.addLayer(spawn.first().select(['agb']).updateMask(spawn.first().select(['agb']).neq(0)), {"min": 1, "max": 100, "palette": biomass_palette}, 'Spawn AGB Density')
# m.addLayer(bio_2021.select(['dpm_t_ha']).clip(y2y), {"min": 1, "max": 60, "palette": biomass_palette}, 'DPM Density')
m.addLayer(pixel_area_agb_extent.clip(y2y), {}, 'Carbon Mask')
m.addLayer(lc.clip(y2y), {"min": 1, "max": 19, "palette": lc_palette}, 'Landcover')

# Display the map
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…